# Parquet: Acelere sua Análise de Dados e Reduza Custos em Nuvem

In [ ]:
# instalar Parquet, Pandas e Pyarrow
%!pip install parquet
%!pip install pandas
%!pip install pyarrow

In [96]:
# importar os pacotes necessários
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

In [97]:
# Criando um DataFrame de exemplo com diversos tipos de dados
df = pd.DataFrame({
    'Nome': ['Maria', 'John', 'Tonico', 'Mariane'],
    'Idade': [25, 30, 35, np.nan],  # Incluindo um valor nulo
    'Salário': [50000.50, 60000.75, 70000.00, 80000.25],
    'Data_Admissão': pd.to_datetime(['2020-01-15', '2019-05-20', '2018-11-01', '2021-03-10']),
    'Descrição': ['Desenvolvedora Python', 'Analista de Dados', 'Cientista de Dados', 'Gerente de Projetos com acentuação çãõ']
})

### PyArrow: Uma Solução Completa de Parquet

In [ ]:
# Converta o DataFrame em uma Arrow Table
df = pa.Table.from_pandas(df)

# Escrevendo o DataFrame para um arquivo Parquet usando pyarrow
try:
    pq.write_table(df, 'dataset/sample.parquet')
    print("\nArquivo Parquet 'dataset/sample.parquet' escrito com sucesso usando pyarrow!")
except Exception as e:
    print(f"Erro ao escrever o arquivo Parquet: {e}")

### Ler arquivo `Parquet` utilizando pyarrow
Pode utilizar o método `read_table`.

In [ ]:
# Lendo o arquivo completo com pyarrow
try:
    # Leia o arquivo Parquet em uma Arrow Table
    df = pq.read_table('dataset/sample.parquet')
    print("\nDataFrame lido completo com pyarrow:")
    print(df)

    # Converte a tabela pyarrow para um DataFrame pandas, se necessário
    # df_pandas = df.to_pandas() # Descomente esta linha se precisar de um DataFrame pandas
    # print("\nDataFrame lido completo com pandas:")
    # print(df_pandas)

except FileNotFoundError:
    print("\nErro: Arquivo Parquet não encontrado. Certifique-se de que o arquivo 'dataset/sample.parquet' ou 'dataset/sample_fastparquet.parquet' existe no diretório atual.")
except pd.errors.ParserError as pe:
    print(f"\nErro de parsing do Parquet: {pe}")
except Exception as e:
    print(f"\nOutro erro ao ler o arquivo Parquet: {e}")

### Ler apenas colunas especificas em uma tabela utilizando pyarrow
Pode utilizar o método `read_table`.

In [ ]:
# Lendo apenas as colunas 'Nome' e 'Salário' com pyarrow
try:
    # Abre o arquivo Parquet
    df_colunas_selecionadas = pq.read_table('dataset/sample.parquet', columns=['Nome', 'Salário'])
    print("\nTabela com colunas selecionadas (Nome e Salário) com pyarrow:")
    print(df_colunas_selecionadas)

    # Utilizando o pandas, se necessário
    # df_colunas_selecionadas_pd = pd.read_parquet('dataset/sample.parquet', columns=['Nome', 'Salário'], engine='pyarrow')
    # print(df_colunas_selecionadas_pd)

except FileNotFoundError:
    print("\nErro: Arquivo Parquet não encontrado. Certifique-se de que o arquivo 'dataset/sample.parquet' existe no diretório atual.")
except Exception as e:
    print(f"\nOutro erro ao ler o arquivo Parquet: {e}")

### Adicionar uma nova coluna e seus valores a uma tabela pyarrow

Pode utilizar o método `add_column`.

In [ ]:
# Criar uma nova coluna com pyarrow
try:
    nova_coluna_nome = 'Status'
    nova_coluna = pa.array(['Ativo', 'Inativo', 'Ativo', 'Inativo'])

    # Verifica se a coluna já existe
    if nova_coluna_nome in df.column_names:
         print(f"\nA coluna '{nova_coluna_nome}' já existe na tabela. Não será adicionada.")
    else:
        # Adiciona a nova coluna à tabela
        df = df.add_column(len(df.column_names), pa.field(nova_coluna_nome, nova_coluna.type), nova_coluna)

        # Converte a tabela pyarrow para um DataFrame pandas para visualização
        df_final = df.to_pandas()

        print("\nTabela com a nova coluna:")
        print(df_final)

        # Escreve a tabela em um arquivo Parquet
        pq.write_table(df, 'dataset/output_com_nova_coluna.parquet')

        print("\nArquivo Parquet 'dataset/output_com_nova_coluna.parquet' escrito com sucesso!")

except Exception as e:
    print(f"\nErro ao adicionar a coluna e escrever o arquivo Parquet: {e}")

### Alterar um valor específico em uma coluna de uma tabela com pyarrow.

Embora o processo seja um pouco diferente de como faríamos em um pandas.DataFrame diretamente. A tabela pyarrow é imutável, então você precisará criar um novo array com a modificação e substituir a coluna inteira.

In [ ]:
# Alterar valor de um indice da coluna com pyarrow
try:

    # Especificações da alteração
    coluna_para_alterar = 'Salário'
    indice_do_valor = 1  # Índice do valor a ser alterado (Bob)
    novo_valor = 65000

    # Verifica se a coluna existe
    if coluna_para_alterar not in df.column_names:
        print(f"\nErro: A coluna '{coluna_para_alterar}' não existe.")
    else:
        # Verifica se o índice está dentro do alcance
        coluna = df.column(coluna_para_alterar)
        if indice_do_valor < 0 or indice_do_valor >= len(coluna):
             print(f"\nErro: Índice '{indice_do_valor}' fora do alcance da coluna '{coluna_para_alterar}'.")
        else:
            # Cria um novo array (cópia) com o valor alterado
            novo_array = coluna.to_numpy().copy() # Usamos copy() aqui
            novo_array[indice_do_valor] = novo_valor
            novo_array = pa.array(novo_array)

            # Substitui a coluna original pela nova coluna
            df = df.set_column(df.column_names.index(coluna_para_alterar), pa.field(coluna_para_alterar, novo_array.type), novo_array)

            print(f"\nValor na coluna '{coluna_para_alterar}' no índice '{indice_do_valor}' alterado para '{novo_valor}' com sucesso.")

            # Converte a tabela para um DataFrame pandas para visualização
            df_final = df.to_pandas()
            print("\nTabela com o valor alterado:")
            print(df_final)


            # Escreve a tabela em um novo arquivo Parquet
            pq.write_table(df, 'dataset/output_valor_alterado.parquet')
            print("\nArquivo Parquet 'dataset/output_valor_alterado.parquet' escrito com sucesso!")


except Exception as e:
    print(f"\nErro ao alterar o valor e escrever o arquivo Parquet: {e}")

### Excluir uma coluna existente de uma tabela com pyarrow.

Pode usar o método` remove_column`

In [ ]:
# Excluir uma coluna com pyarrow

try:
    # Nome da coluna a ser removida
    coluna_a_remover = 'Salário'

    # Verifica se a coluna existe antes de tentar remover
    if coluna_a_remover in df.column_names:
      # Remove a coluna da tabela
        df = df.remove_column(df.column_names.index(coluna_a_remover))
        print(f"\nA coluna '{coluna_a_remover}' foi removida com sucesso.")

      # Converte a tabela pyarrow para um DataFrame pandas para visualização
        df_final = df.to_pandas()

        print("\nTabela sem a coluna:")
        print(df_final)

        # Escreve a tabela em um novo arquivo Parquet
        pq.df(df, 'dataset/output_sem_coluna.parquet')

        print("\nArquivo Parquet 'dataset/output_sem_coluna.parquet' escrito com sucesso!")
    else:
        print(f"\nA coluna '{coluna_a_remover}' não existe na tabela. A remoção não será realizada.")
except Exception as e:
    print(f"\nErro ao remover a coluna e escrever o arquivo Parquet: {e}")

In [ ]:
# Escrevendo o DataFrame para um arquivo Parquet usando pandas
try:
    df.to_parquet('dataset/sample.parquet', engine='pyarrow')
    print("\nArquivo Parquet 'dataset/sample.parquet' escrito com sucesso usando pandas!")
except Exception as e:
    print(f"Erro ao escrever o arquivo Parquet: {e}")

# Lendo apenas as colunas 'Nome' e 'Salário' com pandas
try:
    df_colunas_selecionadas = pd.read_parquet('dataset/sample.parquet', columns=['Nome', 'Salário'], engine='pyarrow')
    print("\nDataFrame com colunas selecionadas (Nome e Salário) com pyarrow:")
    print(df_colunas_selecionadas)

except FileNotFoundError:
    print("\nErro: Arquivo Parquet não encontrado. Certifique-se de que o arquivo 'dataset/sample.parquet' ou 'dataset/sample_fastparquet.parquet' existe no diretório atual.")
except pd.errors.ParserError as pe:
    print(f"\nErro de parsing do Parquet: {pe}")
except Exception as e:
    print(f"\nOutro erro ao ler o arquivo Parquet: {e}")